In [15]:
import SimpleITK as sitk
from scipy.ndimage import label, binary_fill_holes

def save_array_to_nii(mask_array, nii_template_path, nii_save_path):

    sitk_image_object = sitk.ReadImage(nii_template_path)
    output_spacing = sitk_image_object.GetSpacing()
    output_direction = sitk_image_object.GetDirection()
    output_origin = sitk_image_object.GetOrigin()
#     print(output_spacing ,output_direction,output_origin)

    nrrd_output = sitk.GetImageFromArray(mask_array)
    nrrd_output.SetSpacing(output_spacing)
    nrrd_output.SetDirection(output_direction)
    nrrd_output.SetOrigin(output_origin)

    nrrdWriter = sitk.ImageFileWriter()
    nrrdWriter.SetFileName(nii_save_path)
    nrrdWriter.SetUseCompression(True)
    nrrdWriter.Execute(nrrd_output)
    print(nii_save_path ,'saved')
    
def keep_largest_connected_component(mask):
    labeled, num = label(mask)
    if num == 0:
        return mask
    counts = np.bincount(labeled.ravel())
    counts[0] = 0  # ignore background
    largest_label = counts.argmax()
    return binary_fill_holes((labeled == largest_label)).astype(np.uint8)

In [16]:
import os
import glob
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from scipy.ndimage import label, binary_fill_holes


dataset = 'CT_TT'  # 例如 'AFL_MRI' 或 'Bedrest'


base_dir = f'videos/{dataset}'
seed_count = 3
seed_count = 1
seed_count = 5

seed_count = str(seed_count  )
data_dir = os.path.join(base_dir, 'data_in_jpg')
csv_path = os.path.join(data_dir, f'{dataset}_4class_IOU_all_pid_with_full_mapping_{seed_count}seed.csv')
img_dir = os.path.join(base_dir, 'data_in_nii', 'img_in_nii_L4L5')
# sam2_root = os.path.join(data_dir, 'sam2_results_by_pid')
sam2_root = os.path.join(data_dir, 'sam2_results_by_pid_'+seed_count+'seed')
 
# save_seg_dir = os.path.join(base_dir, 'data_in_nii', 'SAM2_auto_seg_nii_3shot')
save_seg_dir = os.path.join(base_dir, 'data_in_nii', 'SAM2_auto_seg_nii_'+seed_count+'shot')

os.makedirs(save_seg_dir, exist_ok=True)

if dataset == 'Bedrest':
    img_dir = os.path.join(base_dir, 'data_in_nii', 'img_in_nii_water_72_L4L5')

class_values = [50, 100, 150, 200]  # 类别对应的灰度值

df = pd.read_csv(csv_path)

for pid, group in tqdm(df.groupby('pid')):
    group = group.sort_values('slice_id').reset_index(drop=True)
    frame_ids = group['frame_idx'].tolist()
    slice_ids = group['slice_id'].tolist()

    img_path_list = glob.glob(os.path.join(img_dir, f"{pid}*.nii.gz"))
    if not img_path_list:
        print(f"❌ Image not found for {pid}")
        continue
    if len(img_path_list) > 1:
        print(f"⚠️ Multiple image files found for {pid}, using the first one.")
    img_path = img_path_list[0]

    img = sitk.ReadImage(img_path)
    img_array = sitk.GetArrayFromImage(img)
    H, W = img_array.shape[1:]  # (slice, height, width)

    sam2_pred_dir = os.path.join(sam2_root, pid, 'SAM2_seg_mask_nolap')

    seg_array = []

    for frame_id, slice_id in zip(frame_ids, slice_ids):
        per_class_masks = []
        for i, class_val in enumerate(class_values):
            png_path = os.path.join(sam2_pred_dir, f"frame_{frame_id}_obj_{i+1}.png")
            if not os.path.exists(png_path):
                print(f"⚠️ Missing mask: {png_path}")
                continue
            mask = cv2.imread(png_path, cv2.IMREAD_GRAYSCALE)
            mask_bin = (np.abs(mask - class_val) < 10).astype(np.uint8)
            mask_clean = keep_largest_connected_component(mask_bin)  # 保留最大连通域
            mask_resized = cv2.resize(mask_clean, (W, H), interpolation=cv2.INTER_NEAREST)
            per_class_masks.append(mask_resized)

        if len(per_class_masks) != len(class_values):
            print(f"❌ Incomplete masks for {pid} frame {frame_id}, skipping this slice")
            continue

        multi_mask = np.zeros((H, W), dtype=np.uint8)
        for cls_idx, binary_mask in enumerate(per_class_masks):
            multi_mask[binary_mask > 0] = cls_idx + 1

        if 'ID' in pid or 'CT_TT' in dataset:
            multi_mask = np.rot90(multi_mask, 2)  # 旋转180度

        seg_array.append(multi_mask)

    if not seg_array:
        print(f"⚠️ No valid segmentation slices for {pid}, skipping saving")
        continue

    seg_stack = np.stack(seg_array)
    for class_idx in range(1, len(class_values) + 1):
        class_mask_3d = (seg_stack == class_idx).astype(np.uint8)
        class_mask_3d_clean = keep_largest_connected_component(class_mask_3d)
        seg_stack[seg_stack == class_idx] = 0
        seg_stack[class_mask_3d_clean > 0] = class_idx


    save_path = os.path.join(save_seg_dir, f"{pid}_sam2_seg.nii.gz")
    save_array_to_nii(seg_stack, img_path, save_path)

  1%|▏                                          | 1/173 [00:00<01:18,  2.20it/s]

videos/CT_TT/data_in_nii/SAM2_auto_seg_nii_5shot/s0010_sam2_seg.nii.gz saved


  1%|▍                                          | 2/173 [00:00<01:11,  2.40it/s]

videos/CT_TT/data_in_nii/SAM2_auto_seg_nii_5shot/s0011_sam2_seg.nii.gz saved


  2%|▋                                          | 3/173 [00:01<01:02,  2.74it/s]

...

videos/CT_TT/data_in_nii/SAM2_auto_seg_nii_5shot/s1415_sam2_seg.nii.gz saved


 99%|████████████████████████████████████████▊| 172/173 [00:54<00:00,  3.65it/s]

videos/CT_TT/data_in_nii/SAM2_auto_seg_nii_5shot/s1423_sam2_seg.nii.gz saved


100%|█████████████████████████████████████████| 173/173 [00:55<00:00,  3.13it/s]

videos/CT_TT/data_in_nii/SAM2_auto_seg_nii_5shot/s1424_sam2_seg.nii.gz saved


In [17]:

def dice_coefficient(pred, gt, class_id):
    pred_bin = (pred == class_id)
    gt_bin = (gt == class_id)
    intersection = np.sum(pred_bin & gt_bin)
    volume_sum = np.sum(pred_bin) + np.sum(gt_bin)
    if volume_sum == 0:
        return 1.0
    return 2 * intersection / volume_sum


base_dir = f'videos/{dataset}'
# save_seg_dir = os.path.join(base_dir, 'data_in_nii', 'SAM2_auto_seg_nii')
label_dir = os.path.join(base_dir, 'data_in_nii', 'label_in_nii_L4L5_4class')
if dataset == 'Bedrest':
    label_dir = os.path.join(base_dir, 'data_in_nii', 'label_in_nii_72_water_72_L4L5')

class_ids = [1, 2,3 ,4]

results = []
for fname in tqdm(os.listdir(save_seg_dir)):
    if not fname.endswith('.nii.gz'):
        continue
    pid = fname.replace('_sam2_seg.nii.gz', '')
    pid = pid.replace('_L4L5_lcc_combined.nii.gz', '')
    pred_path = os.path.join(save_seg_dir, fname)
    print(pid, 'processing')
    label_path = glob.glob(os.path.join(label_dir, f"{pid}*.nii.gz"))[0]
    if not os.path.exists(label_path):
        print(f"⚠️ Label not found for {pid}, skipping.")
        continue

    pred = sitk.GetArrayFromImage(sitk.ReadImage(pred_path))
    gt = sitk.GetArrayFromImage(sitk.ReadImage(label_path))

    dscs = [dice_coefficient(pred, gt, c) for c in class_ids]
    results.append([pid, *dscs])

df = pd.DataFrame(results, columns=['pid'] + [f'dsc_class{c}' for c in class_ids])
df.to_csv(os.path.join(base_dir, f'{dataset}_SAM2_auto_seg_DSC_summary_{seed_count}seed.csv'), index=False)


  5%|██▏                                        | 9/173 [00:00<00:01, 82.75it/s]

s0928 processing
s0835 processing
s1423 processing
s0467 processing
s0862 processing
s0649 processing
s0613 processing
s0650 processing
s1379 processing
s0178 processing
s1321 processing
s0677 processing
s0227 processing
s0777 processing
s0320 processing
s0076 processing
s0546 processing
s1304 processing


 17%|███████                                   | 29/173 [00:00<00:01, 93.03it/s]

s1394 processing
s1283 processing
s0794 processing
s0071 processing
s0878 processing
s0472 processing
s0584 processing
s1293 processing
s0720 processing
s0536 processing
s0762 processing
s0899 processing
s1130 processing
s0970 processing
s0577 processing
s0015 processing
s0196 processing
s0593 processing
s0350 processing
s0078 processing


 28%|███████████▉                              | 49/173 [00:00<00:01, 94.69it/s]

...

s1247 processing
s0442 processing
s1424 processing
s0024 processing
s0332 processing
s0255 processing
s0797 processing
s0287 processing
s0104 processing
s0864 processing
s0476 processing
s0040 processing
s1400 processing
s1249 processing
s1123 processing
s1031 processing
s0014 processing
s0447 processing
s0664 processing


 92%|█████████████████████████████████████▉   | 160/173 [00:01<00:00, 91.69it/s]

s1365 processing
s0778 processing
s0628 processing
s0034 processing
s1411 processing
s0589 processing
s1361 processing
s0053 processing
s1380 processing
s1161 processing
s0201 processing
s1264 processing
s1152 processing
s0428 processing
s0869 processing
s0483 processing
s1099 processing
s1415 processing
s0933 processing


100%|█████████████████████████████████████████| 173/173 [00:01<00:00, 90.82it/s]

s1082 processing
s0096 processing
s0334 processing


In [18]:
print(dataset)
mean_values = df[['dsc_class1', 'dsc_class2']].mean()
std_values = df[['dsc_class1', 'dsc_class2']].std()

print("Mean DSC per class:")
print(mean_values)

print("\nStandard Deviation of DSC per class:")
print(std_values)
print(df.shape)
df.head()

CT_TT
Mean DSC per class:
dsc_class1    0.896914
dsc_class2    0.888824
dtype: float64

Standard Deviation of DSC per class:
dsc_class1    0.029100
dsc_class2    0.021087
dtype: float64
(173, 5)


,pid,dsc_class1,dsc_class2,dsc_class3,dsc_class4
0,s0928,0.909987,0.908044,1.0,1.0
1,s0835,0.881805,0.887637,1.0,1.0
2,s1423,0.910966,0.898941,1.0,1.0
3,s0467,0.848591,0.885151,1.0,1.0
4,s0862,0.899787,0.908861,1.0,1.0


In [8]:
print(dataset)
mean_values = df[['dsc_class1', 'dsc_class2', 'dsc_class3', 'dsc_class4']].mean()
std_values = df[['dsc_class1', 'dsc_class2', 'dsc_class3', 'dsc_class4']].std()

print("Mean DSC per class:")
print(mean_values)

print("\nStandard Deviation of DSC per class:")
print(std_values)

df.head()

CT_TT
Mean DSC per class:
dsc_class1    0.897432
dsc_class2    0.884695
dsc_class3    1.000000
dsc_class4    1.000000
dtype: float64

Standard Deviation of DSC per class:
dsc_class1    0.026783
dsc_class2    0.030179
dsc_class3    0.000000
dsc_class4    0.000000
dtype: float64


,pid,dsc_class1,dsc_class2,dsc_class3,dsc_class4
0,s0928,0.907166,0.908329,1.0,1.0
1,s0835,0.886207,0.890549,1.0,1.0
2,s1423,0.912794,0.896489,1.0,1.0
3,s0467,0.868498,0.881045,1.0,1.0
4,s0862,0.901225,0.912721,1.0,1.0
